In [1]:
import os 
import numpy as np 
import torch 
from PIL import Image 

In [2]:
class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root 
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))
        
    def __getitem__(self, index):
        img_path = os.path.join(self.root, "PNGImages", self.imgs[index])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[index])
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)
        mask = np.array(mask)
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:]
        masks = mask == obj_ids[:, None, None]
        
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.nonzero(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((num_objs), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        
        image_id = torch.tensor([index])
        area = (boxes[:,3]-boxes[:,1]) * (boxes[:,2]- boxes[:,0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        return img, target
    
    def __len__(self):
        return len(self.imgs)


In [3]:
dataset = PennFudanDataset('../QuickStart/data/PennFudanPed/', transforms=None)
dataset[0]

(<PIL.Image.Image image mode=RGB size=559x536>,
 {'boxes': tensor([[159., 181., 301., 430.],
          [419., 170., 534., 485.]]),
  'labels': tensor([1, 1]),
  'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8),
  'image_id': tensor([0]),
  'area': tensor([35358., 36225.]),
  'iscrowd': tensor([0, 0])})

In [4]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [13]:
import utils
import torchvision 
from torchvision.models.detection import retinanet_resnet50_fpn

#model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
model_demo = retinanet_resnet50_fpn(weights="DEFAULT")

In [14]:
dataset = PennFudanDataset('../QuickStart/data/PennFudanPed/', get_transform(train=True))
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle = True, collate_fn=utils.collate_fn
)
images, targets = next(iter(dataloader))
images = list(image for image in images)
targets = [{k:v for k,v in t.items()} for t in targets]
output = model_demo(images, targets)

In [15]:
output

{'classification': tensor(0.4413, grad_fn=<DivBackward0>),
 'bbox_regression': tensor(0.1649, grad_fn=<DivBackward0>)}

In [16]:
model_demo.eval()
x = [torch.rand(3, 300, 400), torch.rand(3,500,400)]
predictions = model_demo(x)

In [17]:
predictions

[{'boxes': tensor([[0.0000e+00, 1.5160e+00, 4.0000e+02, 3.0000e+02],
          [0.0000e+00, 8.6291e+00, 4.0000e+02, 2.9974e+02],
          [1.4005e+00, 0.0000e+00, 4.0000e+02, 3.0000e+02],
          [1.4005e+00, 0.0000e+00, 4.0000e+02, 3.0000e+02],
          [1.4005e+00, 0.0000e+00, 4.0000e+02, 3.0000e+02],
          [0.0000e+00, 2.1225e-01, 4.0000e+02, 3.0000e+02],
          [1.4005e+00, 0.0000e+00, 4.0000e+02, 3.0000e+02]],
         grad_fn=<StackBackward0>),
  'scores': tensor([0.1515, 0.1190, 0.1158, 0.0950, 0.0811, 0.0672, 0.0668],
         grad_fn=<IndexBackward0>),
  'labels': tensor([72, 28,  9,  7,  3, 64, 65])},
 {'boxes': tensor([[  6.8891,   3.4301, 400.0000, 500.0000]], grad_fn=<StackBackward0>),
  'scores': tensor([0.0593], grad_fn=<IndexBackward0>),
  'labels': tensor([72])}]

In [18]:
from engine import train_one_epoch, evaluate 

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

dataset = PennFudanDataset('../QuickStart/data/PennFudanPed/', get_transform(train=True)) 
dataset_test = PennFudanDataset('../QuickStart/data/PennFudanPed/', get_transform(train=False))

indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])


dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=utils.collate_fn)   
dataloader_test =  torch.utils.data.DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=utils.collate_fn)   

In [19]:
import torch.nn 
from torchvision.models.detection import RetinaNet 
from torchvision.models.resnet import resnet50
from torchvision.models.detection import backbone_utils
from torchvision.ops.misc import FrozenBatchNorm2d
from torchvision.ops.feature_pyramid_network import LastLevelP6P7

trainable_backbone_layers = backbone_utils._validate_trainable_layers(
    is_trained=True, trainable_backbone_layers=None, max_value=5, default_value=3
)
norm_layer = FrozenBatchNorm2d
backbone = resnet50(weights="IMAGENET1K_V1", progress=True, norm_layer=norm_layer)
backbone = backbone_utils._resnet_fpn_extractor(
    backbone, trainable_backbone_layers, returned_layers=[2, 3, 4],
    extra_blocks=LastLevelP6P7(256,256)
)
model = RetinaNet(backbone, num_classes=2)


In [22]:
model.eval()

RetinaNet(
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(256, e

In [21]:
model_demo.eval()

RetinaNet(
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(256, eps=0.0)


In [ ]:
import torch
model.to(device=device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params=params, lr=0.005, momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

num_epochs = 10

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, dataloader, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, dataloader_test, device=device)
print("That's it! ")